### Определение инструмента

In [1]:
from dotenv import load_dotenv

load_dotenv()

True

In [10]:
from langchain.tools import tool
from langchain.agents import create_agent
from langchain.messages import HumanMessage
from langchain_openai import ChatOpenAI
import os

In [11]:
OPENROUTER_API_KEY = os.getenv("OPENROUTER_API_KEY")
if not OPENROUTER_API_KEY:
    raise EnvironmentError("Установите OPENROUTER_API_KEY в файле .env")

llm = ChatOpenAI(
    model="google/gemini-2.5-flash",
    base_url="https://openrouter.ai/api/v1",
    api_key=OPENROUTER_API_KEY
)

#### Базовый способ

In [3]:
@tool
def square_root(x: float) -> float:
    """Вычислите квадратный корень из числа"""
    return x ** 0.5

##### Явное задание имени

In [4]:
@tool("square_root")
def tool1(x: float) -> float:
    """Вычислите квадратный корень из числа"""
    return x ** 0.5

##### Явное задание имени + описания

- Теперь и имя, и описание заданы явно.
- Это лучшая практика в production: так ты контролируешь, что видит LLM.
- Особенно важно, если docstring длинный или содержит технические детали, не нужные модели.

In [5]:
@tool("square_root", description="Вычислите квадратный корень из числа")
def tool1(x: float) -> float:
    return x ** 0.5

##### Прямой вызов инструмента
- Инструменты в LangChain поддерживают единый интерфейс invoke.
- Вход — словарь с аргументами (даже если аргумент один!).
- Выход — результат функции (21.61018...).

In [6]:
tool1.invoke({"x": 467})

21.61018278497431

### Добавляем tool в агента

In [ ]:
agent = create_agent(
    model=llm,
    tools=[tool1],
    system_prompt="Вы — волшебник арифметики. Используйте свои инструменты, чтобы вычислить квадратный корень и квадрат любого числа."
)

In [13]:
question = HumanMessage(content="Чему равен квадратный корень из 467?")

response = agent.invoke(
    {"messages": [question]}
)
print(response['messages'][-1].content)

Квадратный корень из 467 равен приблизительно 21,61.


In [14]:
from pprint import pprint

pprint(response['messages'])

[HumanMessage(content='Чему равен квадратный корень из 467?', additional_kwargs={}, response_metadata={}, id='8cfc5c6c-ec22-44f4-9fd7-5946f4879ff4'),
 AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 5, 'prompt_tokens': 54, 'total_tokens': 59, 'completion_tokens_details': {'accepted_prediction_tokens': None, 'audio_tokens': None, 'reasoning_tokens': 0, 'rejected_prediction_tokens': None, 'image_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0, 'video_tokens': 0}, 'cost': 2.87e-05, 'is_byok': False, 'cost_details': {'upstream_inference_cost': None, 'upstream_inference_prompt_cost': 1.62e-05, 'upstream_inference_completions_cost': 1.25e-05}}, 'model_provider': 'openai', 'model_name': 'google/gemini-2.5-flash', 'system_fingerprint': None, 'id': 'gen-1766942713-mneop6MABCJYK1IR1Apn', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--019b65fe-24d2-7491-8905-3539c9f6bc20-0', tool_calls=[

In [15]:
print(response["messages"][1].tool_calls)

[{'name': 'square_root', 'args': {'x': 467}, 'id': 'tool_square_root_Osv31Bvyyp4FDFaIQSul', 'type': 'tool_call'}]
